## Oracle Definition ##

This implements the oracle $D |z\rangle |k\rangle = |z \oplus x_k \rangle |k\rangle$ used in the Grover
search algorithm.

In [1]:
operation DatabaseOracleFromInts(
    markedElement : Int,
    markedQubit: Qubit,
    databaseRegister: Qubit[]
) : Unit {

    body (...) {
        (ControlledOnInt(markedElement, X))(databaseRegister, markedQubit);
    }

    adjoint auto;
    controlled auto;
    adjoint controlled auto;

}

DatabaseOracleFromInts

In [2]:
ControlledOnInt?

Microsoft.Quantum.Canon.ControlledOnInt

## State Preparation ##

This implements an oracle $DU$ that prepares the start state
$DU |0\rangle|0\rangle = 1 / \sqrt{N} |1\rangle|\text{marked}\rangle + \sqrt{N - 1} / \sqrt{N} |0\rangle|\text{unmarked}\rangle$ where $N = 2^n$.

In [3]:
operation PrepareDatabaseRegister(
    markedElement : Int,
    idxMarkedQubit: Int,
    startQubits: Qubit[]
) : Unit {

    body (...) {
        let flagQubit = startQubits[idxMarkedQubit];
        let databaseRegister = Exclude([idxMarkedQubit], startQubits);

        // Apply 𝑈.
        ApplyToEachCA(H, databaseRegister);

        // Apply 𝐷.
        DatabaseOracleFromInts(markedElement, flagQubit, databaseRegister);

    }

    adjoint auto;
    controlled auto;
    adjoint controlled auto;
}


PrepareDatabaseRegister

Here, we wrap our state preparation in a *user-defined type* to indicate that it is a state preparation oracle.

In [4]:
function GroverStatePrepOracle(markedElement : Int) : StateOracle {
    return StateOracle(PrepareDatabaseRegister(markedElement, _, _));
}

GroverStatePrepOracle

## Grover's Algorithm ##

In [5]:
function GroverSearch(
    markedElement: Int,
    nIterations: Int,
    idxMarkedQubit: Int
) : (Qubit[] => Unit : Adjoint, Controlled) {

    return AmpAmpByOracle(nIterations, GroverStatePrepOracle(markedElement), idxMarkedQubit);

}

GroverSearch

In [6]:
operation ApplyQuantumSearch() : (Result, Int) {

    let nIterations = 6;
    let nDatabaseQubits = 6;
    let markedElement = 3;
    
    // Allocate variables to store measurement results.
    mutable resultSuccess = Zero;
    mutable numberElement = 0;

    using ((markedQubit, databaseRegister) = (Qubit(), Qubit[nDatabaseQubits])) {

        // Implement the quantum search algorithm.
        (GroverSearch(markedElement, nIterations, 0))([markedQubit] + databaseRegister);

        // Measure the marked qubit. On success, this should be One.
        set resultSuccess = MResetZ(markedQubit);

        // Measure the state of the database register post-selected on
        // the state of the marked qubit.
        let resultElement = MultiM(databaseRegister);
        set numberElement = PositiveIntFromResultArr(resultElement);

        // These reset all qubits to the |0〉 state, which is required 
        // before deallocation.
        ResetAll(databaseRegister);

    }

    // Returns the measurement results of the algorithm.
    return (resultSuccess, numberElement);
    
}

ApplyQuantumSearch

In [7]:
%simulate ApplyQuantumSearch

(One, 3)

## Epilogue ##

In [8]:
%version

Component    Version
------------ -------------
iqsharp      0.5.1903.2702
Jupyter Core 1.1.12077.0